# Supervisor (Polyglot)

In [ ]:
#!import ../nbs/Testing.dib

In [ ]:
#r @"../../../../../../../.nuget/packages/fsharp.control.asyncseq/3.2.1/lib/netstandard2.1/FSharp.Control.AsyncSeq.dll"
#r @"../../../../../../../.nuget/packages/system.reactive/6.0.1-preview.1/lib/net6.0/System.Reactive.dll"
#r @"../../../../../../../.nuget/packages/system.reactive.linq/6.0.1-preview.1/lib/netstandard2.0/System.Reactive.Linq.dll"
#r @"../../../../../../../.nuget/packages/argu/6.1.1/lib/netstandard2.0/Argu.dll"
#r @"../../../../../../../.nuget/packages/system.commandline/2.0.0-beta4.22272.1/lib/net6.0/System.CommandLine.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.http.connections.common/7.0.0/lib/net7.0/Microsoft.AspNetCore.Http.Connections.Common.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.http.connections.client/7.0.0/lib/net7.0/Microsoft.AspNetCore.Http.Connections.Client.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.signalr.common/7.0.0/lib/net7.0/Microsoft.AspNetCore.SignalR.Common.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.signalr.client/7.0.0/lib/net7.0/Microsoft.AspNetCore.SignalR.Client.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.signalr.client.core/7.0.0/lib/net7.0/Microsoft.AspNetCore.SignalR.Client.Core.dll"
#r @"../../../../../../../.nuget/packages/fsharp.json/0.4.1/lib/netstandard2.0/FSharp.Json.dll"

In [ ]:
#!import ../nbs/Common.fs
#!import ../nbs/CommonFSharp.fs
#!import ../nbs/Async.fs
#!import ../nbs/AsyncSeq.fs
#!import ../nbs/Networking.fs
#!import ../nbs/Runtime.fs
#!import ../nbs/FileSystem.fs

In [ ]:
open Common
open FileSystem.Operators
open Microsoft.AspNetCore.SignalR.Client

## sendJson

In [ ]:
let inline sendJson (port : int) (json : string) = async {
    let! portOpen = Networking.testPortOpen port
    if portOpen then
        try
            let connection = HubConnectionBuilder().WithUrl($"http://127.0.0.1:{port}").Build()
            do! connection.StartAsync () |> Async.AwaitTask
            let! result = connection.InvokeAsync<string>("ClientToServerMsg", json) |> Async.AwaitTask
            do! connection.StopAsync () |> Async.AwaitTask
            trace Debug (fun () -> $"sendJson / port: {port} / json: {json} / result.Length: {result |> Option.ofObj |> Option.map String.length}") getLocals
            return Some result
        with ex ->
            trace Critical (fun () -> $"sendJson / port: {port} / json: {json} / ex: {ex |> printException}") getLocals
            return None
    else
        trace Debug (fun () -> "sendJson / error: port not open") getLocals
        return None
}

## sendObj

In [ ]:
let inline sendObj port obj =
    obj
    |> System.Text.Json.JsonSerializer.Serialize
    |> sendJson port

## awaitCompiler

In [ ]:
type VSCPos = {| line : int; character : int |}
type VSCRange = VSCPos * VSCPos
type RString = VSCRange * string
type TracedError = {| trace : string list; message : string |}
type ClientErrorsRes =
    | FatalError of string
    | TracedError of TracedError
    | PackageErrors of {| uri : string; errors : RString list |}
    | TokenizerErrors of {| uri : string; errors : RString list |}
    | ParserErrors of {| uri : string; errors : RString list |}
    | TypeErrors of {| uri : string; errors : RString list |}

In [ ]:
let inline awaitCompiler port cancellationToken = async {
    let! ct =
        cancellationToken
        |> Option.defaultValue System.Threading.CancellationToken.None
        |> Async.mergeCancellationTokenWithDefaultAsync

    let compiler = MailboxProcessor.Start (fun inbox -> async {
        let! availablePort = Networking.getAvailablePort (Some 60) port
        if port = 13805 && availablePort <> port then
            inbox.Post port
        else
            let repositoryRoot = FileSystem.getSourceDirectory () |> FileSystem.findParent ".paket" false

            let compilerPath =
                repositoryRoot </> "deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release"
                |> System.IO.Path.GetFullPath

            let dllPath = compilerPath </> "Spiral.dll"

            let! exitCode, result =
                Runtime.executeWithOptionsAsync
                    {
                        Command = $@"dotnet ""{dllPath}"" port={availablePort}"
                        CancellationToken = Some ct
                        WorkingDirectory = None
                        OnLine = Some <| fun { Line = line } -> async {
                            if line |> String.contains $"Server bound to: http://localhost:{availablePort}" then
                                do! Networking.waitForPortAccess (Some 10000) true availablePort |> Async.Ignore

                                let rec loop retry = async {
                                    let getLocals () = $"port: {availablePort} / retry: {retry} / {getLocals ()}"
                                    try
                                        let pingObj = {| Ping = true |}
                                        let! pingResult = pingObj |> sendObj availablePort
                                        trace Verbose (fun () -> $"awaitCompiler / Ping / result: {pingResult}") getLocals
                                    with ex ->
                                        trace Verbose (fun () -> $"awaitCompiler / Ping / ex: {ex |> printException}") getLocals
                                        do! Async.Sleep 10
                                        do! loop (retry + 1)
                                }
                                do! loop 0
                                inbox.Post availablePort
                        }
                    }
            trace Debug (fun () -> $"awaitCompiler / exitCode: {exitCode} / result: {result}") getLocals
    }, ct)

    let! serverPort = compiler.Receive ()

    let connection = HubConnectionBuilder().WithUrl($"http://127.0.0.1:{serverPort}").Build ()
    do! connection.StartAsync () |> Async.AwaitTask

    let event = Event<_> ()
    let disposable = connection.On<string> ("ServerToClientMsg", event.Trigger)
    let stream =
        FSharp.Control.AsyncSeq.unfoldAsync
            (fun () -> async {
                let! msg = event.Publish |> Async.AwaitEvent
                return Some (msg |> FSharp.Json.Json.deserialize<ClientErrorsRes>, ())
            })
            ()

    let disposable =
        newDisposable (fun () ->
            disposable.Dispose ()
            connection.StopAsync () |> Async.AwaitTask |> Async.StartImmediate
        )

    return
        serverPort,
        stream,
        disposable
}

## getFileUri

In [ ]:
let inline getFileUri path =
    $"file:///{path |> String.trimStart [| '/' |]}"

In [ ]:
let inline getFilePathFromUri uri =
    match System.Uri.TryCreate (uri, System.UriKind.Absolute) with
    | true, uri -> uri.AbsolutePath |> System.IO.Path.GetFullPath
    | _ -> failwith "invalid uri"

## buildFile

In [ ]:
let inline buildFile timeout cancellationToken path = async {
    let fullPath = path |> System.IO.Path.GetFullPath
    let fileDir = fullPath |> System.IO.Path.GetDirectoryName
    let fileName = fullPath |> System.IO.Path.GetFileNameWithoutExtension
    let! code = fullPath |> FileSystem.readAllTextAsync

    let stream, disposable = FileSystem.watchDirectory true fileDir
    use _ = disposable

    let port =
        if fullPath |> String.startsWith (System.IO.Path.GetTempPath ())
            && fullPath |> String.contains "Microsoft.DotNet.Interactive.App" |> not
        then 13807
        else 13805

    use cts = new System.Threading.CancellationTokenSource ()
    use cts =
        System.Threading.CancellationTokenSource.CreateLinkedTokenSource
            [| cts.Token; cancellationToken |> Option.defaultValue System.Threading.CancellationToken.None |]
    use _ = newDisposable cts.Cancel
    let! serverPort, errors, disposable = awaitCompiler port (Some cts.Token)
    use _ = disposable

    let fsxContentSeq =
        stream
        |> FSharp.Control.AsyncSeq.choose (function
            | _, FileSystem.FileSystemChange.Changed (path, Some content) when path = $"{fileName}.fsx" ->
                Some content
            | _ -> None
        )
        |> FSharp.Control.AsyncSeq.map (fun content ->
            Some (content |> String.replace "\r\n" "\n"), None
        )

    let inline printErrorData (data : {| uri : string; errors : RString list |}) =
        let fileName = data.uri |> System.IO.Path.GetFileName
        let errors =
            data.errors
            |> List.map snd
            |> String.concat "\n"
        $"{fileName}:\n{errors}"

    let errorsSeq =
        errors
        |> FSharp.Control.AsyncSeq.choose (fun error ->
            match error with
            | FatalError message ->
                Some (message, true, error)
            | TracedError data ->
                Some (data.message, true, error)
            | PackageErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, true, error)
            | TokenizerErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, true, error)
            | ParserErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, false, error)
            | TypeErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, true, error)
            | _ -> None
        )
        |> FSharp.Control.AsyncSeq.map (fun (message, fatal, error) ->
            None, Some (message, fatal, error)
        )

    let outputSeq =
        [fsxContentSeq; errorsSeq]
        |> FSharp.Control.AsyncSeq.mergeAll

    let! outputChild =
        ((None, [], None), outputSeq)
        ||> FSharp.Control.AsyncSeq.scan (
            fun (fsxContentResult, errors, firstErrorTicks) (fsxContent, error) ->
                match fsxContent, error with
                | Some fsxContent, None -> Some fsxContent, errors, firstErrorTicks
                | None, Some (message, fatal, error) ->
                    fsxContentResult,
                    (message, error) :: errors,
                    firstErrorTicks
                    |> Option.defaultWith (fun () ->
                        if fatal
                        then System.DateTime.MinValue.Ticks
                        else System.DateTime.Now.Ticks
                    )
                    |> Some
                | _ -> fsxContentResult, errors, firstErrorTicks
        )
        |> FSharp.Control.AsyncSeq.takeWhileInclusive (fun (fsxContent, errors, firstErrorTicks) ->
            let firstErrorMs =
                firstErrorTicks
                |> Option.map (fun t -> System.TimeSpan(System.DateTime.Now.Ticks - t).TotalMilliseconds)
            trace Debug (fun () -> $"buildFile / fsxContent: {fsxContent} / errors: {errors} / firstErrorMs: {firstErrorMs}") getLocals
            match fsxContent, errors, firstErrorMs with
            | None, [], _ -> true
            | None, _, Some firstErrorMs when firstErrorMs < 2000. -> true
            | _ -> false
        )
        |> FSharp.Control.AsyncSeq.map (fun (fsxContent, errors, _) ->
            fsxContent, errors |> List.distinct
        )
        |> FSharp.Control.AsyncSeq.tryLast
        |> Async.catch
        |> Async.runWithTimeoutAsync timeout
        |> Async.StartChild

    let fileOpenObj = {| FileOpen = {| uri = fullPath |> getFileUri; spiText = code |} |}
    let! _fileOpenResult = fileOpenObj |> sendObj serverPort

    let buildFileObj = {| BuildFile = {| uri = fullPath |> getFileUri; backend = "Fsharp" |} |}
    let! _buildFileResult = buildFileObj |> sendObj serverPort

    return!
        outputChild
        |> Async.map (function
            | Some (Ok (Some result)) -> result
            | Some (Error ex) ->
                trace Critical (fun () -> $"buildFile / error: {ex |> printException}") getLocals
                None, []
            | _ -> None, []
        )
}

## persistCode

In [ ]:
let inline persistCode timeout cancellationToken code = async {
    let tempDir = FileSystem.createTempDirectory ()

    let mainPath = tempDir </> "main.spi"
    do! code |> FileSystem.writeAllTextAsync mainPath

    let repositoryRoot = FileSystem.getSourceDirectory () |> FileSystem.findParent ".paket" false

    let spiprojPath = tempDir </> "package.spiproj"
    let spiprojCode =
        $"""packageDir: {repositoryRoot </> "spiral"}
packages:
    |core-
    fsharp
modules:
    main
"""
    do! spiprojCode |> FileSystem.writeAllTextAsync spiprojPath

    let disposable = newDisposable (fun () ->
        ()
        // tempDir |> FileSystem.deleteDirectoryAsync |> Async.Ignore |> Async.RunSynchronously
    )

    return mainPath, disposable
}

## buildCode

In [ ]:
let inline buildCode timeout cancellationToken code = async {
    let! mainPath, disposable = persistCode timeout cancellationToken code
    use _ = disposable
    return! mainPath |> buildFile timeout cancellationToken
}

In [ ]:
//// test

let buildCode timeout cancellationToken code = buildCode timeout cancellationToken code

In [ ]:
//// test

""
|> buildCode 6000 None
|> Async.runWithTimeout 6000
|> _equal None

00:00:00 #1 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230926-2326-4743-4397-47018320563f / result: { CreationTime = 2023-09-26 11:26:47 PM
  Exists = true }
00:00:00 #2 [Debug] runWithTimeoutAsync / timeout: 60
00:00:00 #3 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-984> }
00:00:00 #4 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:00 #5 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:01 #6 [Debug] sendJson / port: 13807 / json: {"Ping":true} / result.Length:
00:00:01 #7 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13807 / retry: 0
00:00:01 #8 [Debug] > Server boun

In [ ]:
//// test

"""inl app () =
    fsharp.console.write_line "text"
    1i32

inl main () =
    app
    |> dyn
    |> ignore
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        Some """let rec closure0 () () : int32 =
    let v0 : string = "text"
    System.Console.WriteLine v0
    1
let v0 : (unit -> int32) = closure0()
()
""",
        []
    )
)

00:00:06 #17 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230926-2326-5384-8433-8443bf73f0f9 / result: { CreationTime = 2023-09-26 11:26:53 PM
  Exists = true }
00:00:06 #18 [Debug] runWithTimeoutAsync / timeout: 60
00:00:06 #19 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-984> }
00:00:06 #20 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:06 #21 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:08 #23 [Debug] sendJson / port: 13807 / json: {"Ping":true} / result.Length:
00:00:08 #24 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13807 / retry: 0
00:00:08 #25 [Debug] > Ser

In [ ]:
//// test

"inl app () =
    0i32

inl a = 1

inl main () =
    app
    |> dyn
    |> ignore
"
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        Some "let rec closure0 () () : int32 =
    0
let v0 : (unit -> int32) = closure0()
()
",
        ["main.spi:
Global inl/let statements should all return functions known at parse time."]
    )
)

00:00:11 #32 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230926-2326-5917-1780-1692e4857f94 / result: { CreationTime = 2023-09-26 11:26:59 PM
  Exists = true }
00:00:11 #33 [Debug] runWithTimeoutAsync / timeout: 60
00:00:11 #34 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-984> }
00:00:12 #35 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:12 #36 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:14 #37 [Debug] sendJson / port: 13807 / json: {"Ping":true} / result.Length:
00:00:14 #38 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13807 / retry: 0
00:00:14 #39 [Debug] > Ser

In [ ]:
//// test

"""inl main () =
    1i32 / 0i32
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        ["An attempt to divide by zero has been detected at compile time."]
    )
)

00:00:16 #48 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230926-2327-0403-0357-018705d50ab8 / result: { CreationTime = 2023-09-26 11:27:04 PM
  Exists = true }
00:00:16 #49 [Debug] runWithTimeoutAsync / timeout: 60
00:00:16 #50 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-984> }
00:00:16 #51 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:16 #52 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:18 #53 [Debug] sendJson / port: 13807 / json: {"Ping":true} / result.Length:
00:00:18 #54 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13807 / retry: 0
00:00:18 #55 [Debug] > Ser

In [ ]:
//// test

"""union a =
    | B
    | c

inl main () =
    ()
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        Some "()
",
        ["main.spi:
Expected: uppercase variable"]
    )
)

00:00:20 #62 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230926-2327-0763-6350-65ee0f4a8f7e / result: { CreationTime = 2023-09-26 11:27:07 PM
  Exists = true }
00:00:20 #63 [Debug] runWithTimeoutAsync / timeout: 60
00:00:20 #64 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-984> }
00:00:20 #65 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:20 #66 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:21 #67 [Debug] sendJson / port: 13807 / json: {"Ping":true} / result.Length:
00:00:21 #68 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13807 / retry: 0
00:00:21 #69 [Debug] > Ser

In [ ]:
//// test

"""
/// abc
inl main () =
    ()
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        ["main.spi:
Expected: whitespace"]
    )
)

00:00:23 #78 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230926-2327-1126-2648-2f6debe765b8 / result: { CreationTime = 2023-09-26 11:27:11 PM
  Exists = true }
00:00:23 #79 [Debug] runWithTimeoutAsync / timeout: 60
00:00:23 #80 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-984> }
00:00:24 #81 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:24 #82 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:25 #83 [Debug] sendJson / port: 13807 / json: {"Ping":true} / result.Length:
00:00:25 #84 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13807 / retry: 0
00:00:25 #85 [Debug] > Ser

In [ ]:
//// test

"""
inl main () =
    real
        inl real_unbox forall a. (obj : a) : a =
            typecase obj with
            | _ => obj
        real_unbox ()
    ()
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        ["Cannot apply a forall with a term."]
    )
)

00:00:26 #91 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230926-2327-1403-0332-0e1b869789c1 / result: { CreationTime = 2023-09-26 11:27:14 PM
  Exists = true }
00:00:26 #92 [Debug] runWithTimeoutAsync / timeout: 60
00:00:26 #93 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-984> }
00:00:27 #94 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:27 #95 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:28 #96 [Debug] sendJson / port: 13807 / json: {"Ping":true} / result.Length:
00:00:28 #97 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13807 / retry: 0
00:00:28 #98 [Debug] > Ser

In [ ]:
//// test

"""
inl main () =
    real
        inl real_unbox forall a. (obj : a) : a =
            typecase obj with
            | _ => obj
        real_unbox `i32 1
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        ["The main function should not have a forall."]
    )
)

00:00:30 #105 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230926-2327-1795-9594-9385c4c5d704 / result: { CreationTime = 2023-09-26 11:27:17 PM
  Exists = true }
00:00:30 #106 [Debug] runWithTimeoutAsync / timeout: 60
00:00:30 #107 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-984> }
00:00:30 #108 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:30 #109 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:32 #110 [Debug] sendJson / port: 13807 / json: {"Ping":true} / result.Length:
00:00:32 #111 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13807 / retry: 0
00:00:32 #112 [Debu

In [ ]:
//// test

"""
inl init_series start end inc =
    inl total : f64 = conv ((end - start) / inc) + 1
    listm.init total (conv >> (*) inc >> (+) start) : list f64

type integration = (f64 -> f64) -> f64 -> f64 -> f64

inl integral dt : integration =
    fun f a b =>
        init_series (a + dt / 2) (b - dt / 2) dt
        |> listm.map (f >> (*) dt)
        |> listm.fold (+) 0

inl main () =
    integral 0.1 (fun x => x ** 2) 0 1
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        Some "0.3325000000000001\n",
        []
    )
)

00:00:35 #119 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230926-2327-2298-9878-96e9d7b6ab22 / result: { CreationTime = 2023-09-26 11:27:22 PM
  Exists = true }
00:00:35 #120 [Debug] runWithTimeoutAsync / timeout: 60
00:00:35 #121 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-984> }
00:00:36 #122 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:36 #123 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:37 #124 [Debug] sendJson / port: 13807 / json: {"Ping":true} / result.Length:
00:00:37 #125 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13807 / retry: 0
00:00:37 #126 [Debu

In [ ]:
//// test

"""
inl init_series start end inc =
    inl total : f64 = conv ((end - start) / inc) + 1
    listm.init total (conv >> (*) inc >> (+) start) : list f64

type integration = (f64 -> f64) -> f64 -> f64 -> f64

inl integral dt : integration =
    fun f a b =>
        init_series (a + dt / 2) (b - dt / 2) dt
        |> listm.map (f >> (*) dt)
        |> listm.fold (+) 0

inl main () =
    integral 0.01 (fun x => x ** 2) 0 1
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
// |> _equal None
|> fun x -> printfn $"{x.ToDisplayString ()}"

00:00:39 #133 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230926-2327-2717-1739-1d913395c3ec / result: { CreationTime = 2023-09-26 11:27:27 PM
  Exists = true }
00:00:39 #134 [Debug] runWithTimeoutAsync / timeout: 60
00:00:39 #135 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-984> }
00:00:40 #136 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:40 #137 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:41 #138 [Debug] sendJson / port: 13807 / json: {"Ping":true} / result.Length:
00:00:41 #139 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13807 / retry: 0
00:00:41 #140 [Debu

## getFileTokenRange

In [ ]:
let inline getFileTokenRange timeout cancellationToken path = async {
    let fullPath = path |> System.IO.Path.GetFullPath
    let! code = fullPath |> FileSystem.readAllTextAsync
    let lines = code |> String.split [| '\n' |]

    let port =
        if fullPath |> String.startsWith (System.IO.Path.GetTempPath ())
            && fullPath |> String.contains "Microsoft.DotNet.Interactive.App" |> not
        then 13807
        else 13805

    let! serverPort, outputs, disposable = awaitCompiler port cancellationToken
    use _ = disposable

    let fileOpenObj = {| FileOpen = {| uri = fullPath |> getFileUri; spiText = code |} |}
    let! fileOpenResult = fileOpenObj |> sendObj serverPort

    let fileTokenRangeObj =
        {|
            FileTokenRange =
                {|
                    uri = fullPath |> getFileUri
                    range =
                        [|
                            {| line = 0; character = 0 |}
                            {| line = lines.Length - 1; character = lines.[lines.Length - 1].Length |}
                        |]
                |}
        |}
    let! fileTokenRangeResult = fileTokenRangeObj |> sendObj serverPort

    return fileTokenRangeResult |> Option.map FSharp.Json.Json.deserialize<int array>
}

## getCodeTokenRange

In [ ]:
let inline getCodeTokenRange timeout cancellationToken code = async {
    let! mainPath, disposable = persistCode timeout cancellationToken code
    use _ = disposable
    return! mainPath |> getFileTokenRange timeout cancellationToken
}

In [ ]:
//// test

"""inl main () = ()"""
|> getCodeTokenRange 10000 None
|> Async.runWithTimeout 10000
|> Option.flatten
|> _equal (Some [|0; 0; 3; 7; 0; 0; 4; 4; 0; 0; 0; 5; 1; 8; 0; 0; 1; 1; 8; 0; 0; 2; 1; 4; 0; 0;
2; 1; 8; 0; 0; 1; 1; 8; 0|])

00:01:02 #553 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230926-2327-5039-3912-393a605dc31c / result: { CreationTime = 2023-09-26 11:27:50 PM
  Exists = true }
00:01:03 #554 [Debug] runWithTimeoutAsync / timeout: 60
00:01:03 #555 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:it@4-372> }
00:01:03 #556 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:01:03 #557 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:01:04 #558 [Debug] sendJson / port: 13807 / json: {"Ping":true} / result.Length:
00:01:04 #559 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13807 / retry: 0
00:01:04 #560 [Debug] > Se

In [ ]:
//// test

"""inl main () = 1i32"""
|> getCodeTokenRange 10000 None
|> Async.runWithTimeout 10000
|> Option.flatten
|> _equal (Some [|0; 0; 3; 7; 0; 0; 4; 4; 0; 0; 0; 5; 1; 8; 0; 0; 1; 1; 8; 0; 0; 2; 1; 4; 0; 0;
2; 1; 3; 0; 0; 1; 3; 12; 0|])

00:01:09 #563 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230926-2327-5661-6172-66fa68cc7df0 / result: { CreationTime = 2023-09-26 11:27:56 PM
  Exists = true }
00:01:09 #564 [Debug] runWithTimeoutAsync / timeout: 60
00:01:09 #565 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:it@4-737> }
00:01:09 #566 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:01:09 #567 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:01:10 #568 [Debug] sendJson / port: 13807 / json: {"Ping":true} / result.Length:
00:01:10 #569 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13807 / retry: 0
00:01:10 #570 [Debug] > Se

## Arguments

In [ ]:
[<RequireQualifiedAccess>]
type Arguments =
    | [<Argu.ArguAttributes.Mandatory>] BuildFile of string * string
    | Timeout of int

    interface Argu.IArgParserTemplate with
        member s.Usage =
            match s with
            | BuildFile _ -> nameof Arguments.BuildFile
            | Timeout _ -> nameof Arguments.Timeout

## main

In [ ]:
let main args =
    let argsMap = args |> Runtime.parseArgsMap<Arguments>

    let inputPath, outputPath =
        match argsMap.[nameof Arguments.BuildFile] with
        | [ Arguments.BuildFile (inputPath, outputPath) ] -> Some (inputPath, outputPath)
        | _ -> None
        |> Option.get

    let timeout =
        match argsMap |> Map.tryFind (nameof Arguments.Timeout) with
        | Some [ Arguments.Timeout timeout ] -> timeout
        | _ -> 30000

    async {
        let! outputCode, errors = inputPath |> buildFile timeout None
        
        errors
        |> List.map snd
        |> List.iter (fun error ->
            trace Critical (fun () -> $"main / error: {error}") getLocals
        )

        match outputCode with
        | Some outputCode ->
            do! outputCode |> FileSystem.writeAllTextAsync outputPath
            return 0
        | None ->
            return 1
    }
    |> Async.runWithTimeout timeout
    |> Option.defaultValue 1

In [ ]:
//// test

let args =
    System.Environment.GetEnvironmentVariable "ARGS"
    |> Runtime.splitArgs
    |> Seq.toArray

match args with
| [||] -> 0
| args -> if main args = 0 then 0 else failwith "main failed"

0